# Agent Sandbox Documentation

## Description

Agent Sandbox is a secure code execution environment that provides AI Agents with safe, isolated runtime spaces. The sandbox supports multiple programming languages including Python, JavaScript, TypeScript, Java, and R, and offers complete file system operations, terminal command execution, and more.

### Key Features

- **Lifecycle Management**: Full lifecycle management including sandbox creation, listing, and termination
- **Multi-language Support**: Supports Python, JavaScript, TypeScript, Java, R, Bash, and more
- **Code Execution Isolation**: Isolation of different code execution environments through context mechanism
- **File System Operations**: Complete file operations including upload, download, create, delete, and rename
- **Terminal Command Execution**: Supports foreground and background command execution
- **Streaming Output**: Real-time streaming output of code execution results
- **File Monitoring**: File system change monitoring with file operation events


### Use Cases

- AI Agent code execution environment
- Code testing and debugging
- Data processing and analysis
- Automated script execution
- Interactive programming environment


## Install Dependencies

In [ ]:
%pip install e2b_code_interpreter

## Sandbox Lifecycle Operations
### Create Sandbox

In [ ]:
import os
from e2b_code_interpreter import Sandbox
# Get API Key and available domain from Agent Sandbox console
# Can be set via environment variables or modified directly here
if not os.getenv("E2B_DOMAIN"):
    os.environ["E2B_DOMAIN"] = "tencentags.com"
if not os.getenv("E2B_API_KEY"):
    os.environ["E2B_API_KEY"] = "your_api_key"

# Before using sandbox service, create a sandbox tool in the console, e.g., "code-interpreter-v1"
# Create code sandbox, default runtime is 5 minutes, use timeout parameter to specify runtime, here we keep it running for 3600s (1h)
sandbox = Sandbox.create(template="code-interpreter-v1",timeout=3600)

### List All Sandboxes

In [ ]:
# Get sandbox list paginator, 5 sandboxes per page
paginator = Sandbox.list(limit=(5))
while paginator.has_next:
    # Print sandboxes in current page
    for sandbox_info in paginator.next_items():
        print(sandbox_info)
    print("next 5 sandbox")

### Reset Expiration Time

In [ ]:
print(sandbox.get_info().started_at, sandbox.get_info().end_at)
sandbox.set_timeout(600)
print(sandbox.get_info().started_at, sandbox.get_info().end_at)

### Connect to Existing Sandbox

In [ ]:
sandbox_2 = Sandbox.connect(sandbox.sandbox_id)
print(sandbox.get_host(8080), sandbox_2.get_host(8080))

### Close Sandbox

In [ ]:
# Immediately close the sandbox
sandbox.kill()

## Sandbox Code Execution Operations

In [ ]:
import os
import asyncio
from e2b_code_interpreter import Sandbox

# Can be set via environment variables or modified directly here
if not os.getenv("E2B_DOMAIN"):
    os.environ["E2B_DOMAIN"] = "tencentags.com"
if not os.getenv("E2B_API_KEY"):
    os.environ["E2B_API_KEY"] = "your_api_key"

sandbox = Sandbox.create(template="code-interpreter-v1",timeout=3600)

### Execute Python Code
Code execution supports streaming output

In [ ]:
python_code = """
import time
print("hello python")
time.sleep(2)
print("hello python after 2 sec")
"""
# Execute the above python code, stream output and print, execution timeout is 600 seconds
print(sandbox.run_code(python_code,on_stdout=lambda data:print(data),timeout=600))

### Execute JavaScript & TypeScript Code

In [ ]:
js_code = """
console.log("hello javascript")
"""
print(sandbox.run_code(js_code,"js"))
ts_code = """
var text:string = "hello typescript"
console.log(text)
"""
print(sandbox.run_code(ts_code,"ts"))

### Execute Java Code

In [ ]:
java_code = """
System.out.println("hello java");
"""
print(sandbox.run_code(java_code,"java"))

### Execute R Code

In [ ]:
r_code = """
print("hello r")
"""
print(sandbox.run_code(r_code,"r"))

### Execute Bash Code

In [ ]:
bash_code = """
echo hello bash
"""
print(sandbox.run_code(bash_code,"bash"))

### Create New Context
Context is used for code execution isolation. If no context is specified, code of the same language runs in the same context, sharing variables and other content.

In [ ]:
ctx = sandbox.create_code_context(language="python")
# Execute code in default context, assign variable x to 1
sandbox.run_code("x=1")
# Execute code in ctx context, assign variable x to 2
sandbox.run_code("x=2",context=ctx)
# Print value of variable x in default context
print(sandbox.run_code("print(x)").logs.stdout[0],end="")
# Print value of variable x in ctx context
print(sandbox.run_code("print(x)",context=ctx).logs.stdout[0],end="")


## Sandbox File System Operations

### Upload File

In [ ]:
import tempfile
# Create temporary file with content "test file"
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.txt') as temp_file:
    temp_file.write("test file")
    temp_file_path = temp_file.name

# Read temporary file and upload to sandbox
with open(temp_file_path, "r") as f:
    sandbox.files.write("temp.txt", f)
print("File uploaded to sandbox: temp.txt")
# Print content of temp.txt in sandbox
print(sandbox.files.read("temp.txt"))
sandbox.files.remove("temp.txt")

### Download File

In [ ]:
# Write temp file to temp.txt
sandbox.files.write("temp.txt","temp file")
# Write content of temp.txt from sandbox to local temp.txt
# After writing, you can see temp.txt in the local current folder
with open("temp.txt", "wb") as local_file:
    local_file.write(sandbox.files.read("temp.txt",format="bytes"))
print("File downloaded to local: temp.txt")
sandbox.files.remove("temp.txt")

### Check If File Exists

In [ ]:
# Currently temp.txt does not exist
print(sandbox.files.exists("temp.txt"))
# Create and write temp.txt file
sandbox.files.write("temp.txt","temp file")
# Now temp.txt exists
print(sandbox.files.exists("temp.txt"))
sandbox.files.remove("temp.txt")


### Rename File

In [ ]:
# Create a file named temp.txt in test directory
sandbox.files.write("./test/temp.txt","temp file")
# Print all files in test directory
for item in sandbox.files.list("./test"):
    print(item.path)
# Rename temp.txt to tempfile.txt
response = sandbox.files.rename("./test/temp.txt","./test/tempfile.txt")
# Print all files in test directory
for item in sandbox.files.list("./test"):
    print(item.path)
sandbox.files.remove("./test")

### Create Directory

In [ ]:
# Print all files in home directory
for item in sandbox.files.list("."):
    print(item.name,end=" ")
# Create a directory named test
response = sandbox.files.make_dir("./test")
print()
# Print all files in home directory
for item in sandbox.files.list("."):
    print(item.name,end=" ")
sandbox.files.remove("./test")

### Monitor File Changes

In [ ]:
# Create a file watcher
watch_handle = sandbox.files.watch_dir(".")
# Create tempfile.txt and write content
sandbox.files.write("tempfile.txt","temp file")
# Print all events during this period
for event in watch_handle.get_new_events():
    print(event)
# Stop file watcher
watch_handle.stop()
sandbox.files.remove("tempfile.txt")
    

## Sandbox PTY Terminal Command Execution

### Execute Terminal Command

In [ ]:
# Execute echo hello using terminal
hello_result = sandbox.commands.run("echo hello")
print(hello_result.stdout)

### Execute Command in Background

In [ ]:
# Use terminal to print 1 2 3, with 1 second interval between each print
echo_code = """
for i in {1..3}; do
    echo -n $i
    sleep 1
done 
echo done
"""
# Execute command in background
echo_handler = sandbox.commands.run(echo_code,background=True)

### List Background Running Commands

In [ ]:
command_list = sandbox.commands.list()
print(command_list)

### Wait for Background Command to Complete with Streaming Output

In [ ]:
response = echo_handler.wait(on_stdout=lambda data:print(data,end=" "),on_stderr=lambda data:print(data,end=" "))

### Send Input to Background Running Command

In [ ]:
# Run code using terminal, waiting for user input
read_code = """
echo wait for user input
read test
echo $test
"""

read_handler = sandbox.commands.run(read_code,background=True,timeout=5)


# Create a task to wait for command completion and display output
wait_task = asyncio.create_task(
    asyncio.to_thread(
        read_handler.wait,
        on_stdout=lambda data: print(data, end=""),
        on_stderr=lambda data: print(data, end="")
    )
)

# Send stdin after 2 seconds delay
await asyncio.sleep(2)
sandbox.commands.send_stdin(read_handler.pid, "input after 2 seconds\n")

# Wait for command completion
response = await wait_task


### Execute Command in Background and Kill It

In [ ]:
# Execute commands in background using terminal, each command sleeps for 10 seconds
sandbox.commands.run("sleep 10",background=True)
sandbox.commands.run("sleep 10",background=True)
# Print currently running commands
command_list = sandbox.commands.list()
print(command_list)
# Kill all running commands
for command in command_list:
    sandbox.commands.kill(command.pid)
# Print currently running commands
command_list = sandbox.commands.list()
print(command_list)

### Connect to Running Command

In [ ]:
read_code = """
echo wait for user input
read test
echo $test
"""
# Execute command in background using terminal
sandbox.commands.run(read_code,background=True,timeout=5)
# Get pid of the last executed command
connect_pid = sandbox.commands.list()[0].pid
# Connect to the last executed command
connect_handler = sandbox.commands.connect(connect_pid)
# Send input to the running command
sandbox.commands.send_stdin(connect_pid,"test_data\n")
# Print command output (output after connection)
print(connect_handler.wait().stdout)


## Simple Data Analysis Demo

### Generate and Upload Test Data
Locally simulate sales data generation and upload to sandbox via file system operations

In [ ]:
import pandas as pd
import numpy as np
import io

# Randomly generate 5000 sales records
np.random.seed(42)
data = {
    'product': np.random.choice(['A', 'B', 'C', 'D', 'E'], 5000),
    'sales': np.random.normal(150, 50, 5000).round(2),
    'region': np.random.choice(['North', 'South', 'East', 'West'], 5000, p=[0.4, 0.3, 0.2, 0.1]),
    'date': pd.date_range('2024-01-01', periods=5000, freq='h')
}
df = pd.DataFrame(data)

# Upload to sandbox
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)
sandbox.files.write("sales_data.csv", csv_buffer.getvalue())



### Execute Code in Sandbox to Analyze Data
Simulate LLM generating code to analyze data in sandbox

In [ ]:
analysis_code = """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('sales_data.csv')
print(f"Data count: {len(df)} records")

# Statistical analysis
product_stats = df.groupby('product')['sales'].agg(['count', 'sum', 'mean']).round(2)
print("\\nProduct Sales Statistics:")
print(product_stats)

# Generate charts
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
product_stats['sum'].plot(kind='bar')
plt.title('Total Sales by Product')

plt.subplot(1, 3, 2)
df.groupby('region')['sales'].sum().plot(kind='pie', autopct='%1.1f%%')
plt.title('Sales by Region')

plt.subplot(1, 3, 3)
df['sales'].hist(bins=30, alpha=0.7)
plt.title('Sales Distribution')
plt.xlabel('Sales')

plt.tight_layout()
plt.savefig('analysis_result.png', dpi=100, bbox_inches='tight')
"""

result = sandbox.run_code(analysis_code)
print(result.logs.stdout[0])


### Read Visualization Analysis Results

In [ ]:
from IPython.display import Image, display

image_data = sandbox.files.read("analysis_result.png", format="bytes")
display(Image(image_data))

### Destroy Sandbox
Temporary files in sandbox will be automatically destroyed

In [ ]:
sandbox.kill()